#### generate import file for kit table

In [1360]:
# home
path = 'C:\\Users\\gratten\\Documents\\python\\projects\\machine_data'

# work
# path = 'C:\\Users\\WDG1DCR\Desktop\\my_projects\\web\\machine_data'

import os
os.chdir(f'{path}\\notebooks')

In [1361]:
import pandas as pd
import numpy as np

In [1362]:
fileName = 'endload_serialnums'

In [1363]:
xl = pd.ExcelFile(f'source_data/{fileName}.xlsx')
df = xl.parse('Enterprise')

In [1364]:
# df = df.head(10)

In [1365]:
serialnums = df['SERIAL #'].to_list()

In [1366]:
kit_dict = {}
for num in range(len(df['SERIAL #'])):
    kit_list = str(df['Carton Size'][num]).split('\n')
    kit_dict[df['SERIAL #'][num]] = kit_list

In [1367]:
kit_columns = ['project',
                'serialnum',
                'customer',
                'speed',
                'size']

In [1368]:
df_kit = pd.DataFrame(columns=kit_columns)

In [1369]:
for k, v in kit_dict.items():
    df_trimmed = df[df['SERIAL #'] == k]

    project = df_trimmed.iloc[:,0].to_string(index=False)
    customer = df_trimmed.iloc[:,2].to_string(index=False)
    speed = df_trimmed.iloc[:,8].to_string(index=False)
 
    kit_columns = ['project',
                'serialnum',
                'customer',
                'speed',
                'size']
    
    new_kits_df = pd.DataFrame(columns=kit_columns)

    
    new_kits_df['size'] = v
    new_kits_df['serialnum'] = k
    new_kits_df['project'] = project
    new_kits_df['customer'] = customer
    new_kits_df['speed'] = speed
    
    df_kit = df_kit.append(new_kits_df)

In [1370]:
df_kit.reset_index(inplace=True, drop=True)

In [1371]:
def decypher(size_string):
    
    size_string = re.sub(r'\"', '', size_string)
    size_string = re.sub(r'[a-vy-zA-VY-Z]', '', size_string)
    size_string = re.sub(r'\)', '', size_string)
    size_string = re.sub(r'\'', '', size_string)
    
    dims = re.split('x|X', size_string)
    
    fraction_pattern = r'(-?\d+)\/(\d+)'
    
    size_dict = {}
    if len(dims) == 3:
        size_dict['length'] = dims[0]
        size_dict['width'] = dims[1]
        size_dict['depth'] = dims[2]
        
    for k, v in size_dict.items():
        
        fraction = re.findall(fraction_pattern, v)
        whole_num = re.sub(fraction_pattern, '', v)
        
        digits_decimals_only = r'[^\d\.]'
        whole_num = re.sub(digits_decimals_only, '', whole_num)
        
        whole_num = float(whole_num)
        
        float_result = 0
        
        if len(fraction) > 0:
            num = fraction[0][0]
            denom = fraction[0][1]
            num = re.sub(r'\D', '', num)
            denom = re.sub(r'\D', '', denom)
            float_result = float(num) / float(denom)
        
        size_dict[k] = whole_num + float_result

    length = 0
    width = 0
    depth = 0

    if len(size_dict) == 3:
        length = size_dict['length']
        width = size_dict['width']
        depth = size_dict['depth']
        
    return length, width, depth

In [1372]:
import re

df_kit['length'] = 0
df_kit['width'] = 0
df_kit['depth'] = 0

for num in range(len(df_kit['size'])):
    size_string = df_kit['size'][num]
    length, width, depth = decypher(size_string)
    
    df_kit.iloc[num, 5] = length
    df_kit.iloc[num, 6] = width
    df_kit.iloc[num, 7] = depth

In [1373]:
df_kit.drop(columns=['size'], inplace=True)

In [1374]:
df_kit_trimmed = df_kit[df_kit['length'] == 0]

In [1375]:
df_kit_trimmed.shape

(4, 7)

In [1376]:
df_kit.head(50)

,project,serialnum,customer,speed,length,width,depth
0,HS5062,ENT-ABOC-101,SUGAR FOODS,NaN,0.0000,0.0000,0.0000
1,HS5077,ENT-103,NESTLE,210CPM,5.8750,1.2500,7.3200
2,HS5077,ENT-103,NESTLE,210CPM,5.8750,1.0625,7.3200
3,HS5097,ENT-104,FURLANI'S,"120CPM (A &B), 110CPM (C & D) & 60CPM (D)",5.2500,2.5000,9.0000
4,HS5097,ENT-104,FURLANI'S,"120CPM (A &B), 110CPM (C & D) & 60CPM (D)",5.2500,3.2500,9.0000
5,HS5097,ENT-104,FURLANI'S,"120CPM (A &B), 110CPM (C & D) & 60CPM (D)",5.2500,3.5000,9.0000
6,HS5097,ENT-104,FURLANI'S,"120CPM (A &B), 110CPM (C & D) & 60CPM (D)",7.1250,4.0000,9.5000
7,HS5097,ENT-104,FURLANI'S,"120CPM (A &B), 110CPM (C & D) & 60CPM (D)",5.2500,3.2500,7.5000
8,HS5103,ENT-105,JOHNSONVILLE,130CPM,5.2500,2.5000,9.7500
9,HS5103,ENT-105,JOHNSONVILLE,130CPM,10.0000,2.7500,9.7500


In [1377]:
# required so that long strings are not truncated when exported to csv
pd.set_option("display.max_colwidth", 10000)

In [1378]:
# os.chdir(f'{path}\\external')
# df_kit.to_csv('endload_kits.csv', sep=';', index=False, header=False)